In [9]:
import csv
import pandas as pd

FIELDNAMES = ['Headline', 'Body ID', 'Stance']
Labels = ['agree', 'disagree', 'discuss', 'unrelated']

def read_bodies(path):
    df = pd.read_csv(path)
    print(df.head)
    Dict = {}
    for index, row in df.iterrows():
        Dict[int(row['Body ID'])] = index
    #print(len(Dict))
    return df,Dict

def read_stances(path):
    df = pd.read_csv(path)
    di = { Labels[0]:0, Labels[1]:1, Labels[2]:2, Labels[3]:3 }
    #print(df.head)
    df['Stance'].replace(di, inplace=True)
    print(df.head)
    return df

trainBodiesPath = "../fnc-1-baseline-master/fnc-1/train_bodies.csv"
trainStancesPath = "../fnc-1-baseline-master/fnc-1/train_stances.csv"
testBodiesPath = "../fnc-1-baseline-master/fnc-1/competition_test_bodies.csv"
testStancesPath = "../fnc-1-baseline-master/fnc-1/competition_test_stances.csv"
train_articles_df,train_dic_articleId_index = read_bodies(trainBodiesPath)
train_stances_df = read_stances(trainStancesPath)
train_labels = train_stances_df[['Stance']].values

test_articles_df,test_dic_articleId_index = read_bodies(testBodiesPath)
test_stances_df = read_stances(testStancesPath)
test_labels = test_stances_df[['Stance']].values

print(train_labels.shape)
print(test_labels.shape)

<bound method NDFrame.head of       Body ID                                        articleBody
0           0  A small meteorite crashed into a wooded area i...
1           4  Last week we hinted at what was to come as Ebo...
2           5  (NEWSER) – Wonder how long a Quarter Pounder w...
3           6  Posting photos of a gun-toting child online, I...
4           7  At least 25 suspected Boko Haram insurgents we...
...       ...                                                ...
1678     2528  Intelligence agencies hunting for identity of ...
1679     2529  While Daleks "know no fear" and "must not fear...
1680     2530  More than 200 schoolgirls were kidnapped in Ap...
1681     2531  A Guantanamo Bay prisoner released last year a...
1682     2532  ANN ARBOR, Mich. – A pizza delivery man in Mic...

[1683 rows x 2 columns]>
<bound method NDFrame.head of                                                 Headline  Body ID  Stance
0      Police find mass graves with at least '15 bodi...    

In [10]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

stopWords = [
        "a", "about", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along",
        "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "another",
        "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", "around", "as", "at", "back", "be",
        "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "behind", "being",
        "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom", "but", "by", "call", "can", "co",
        "con", "could", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight",
        "either", "eleven", "else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone",
        "everything", "everywhere", "except", "few", "fifteen", "fifty", "fill", "find", "fire", "first", "five", "for",
        "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had",
        "has", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself",
        "him", "himself", "his", "how", "however", "hundred", "i", "ie", "if", "in", "inc", "indeed", "interest",
        "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made",
        "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much",
        "must", "my", "myself", "name", "namely", "neither", "nevertheless", "next", "nine", "nobody", "now", "nowhere",
        "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours",
        "ourselves", "out", "over", "own", "part", "per", "perhaps", "please", "put", "rather", "re", "same", "see",
        "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some",
        "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take",
        "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby",
        "therefore", "therein", "thereupon", "these", "they", "thick", "thin", "third", "this", "those", "though",
        "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve",
        "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what",
        "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon",
        "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will",
        "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves"
        ]
train_all_articles_list = []
test_all_articles_list = []

train_all_articles_list = train_articles_df['articleBody'].to_list()
test_all_articles_list = test_articles_df['articleBody'].to_list()
    
train_headlines_list = train_stances_df['Headline'].to_list()
test_headlines_list = test_stances_df['Headline'].to_list()
#Headlines_list[:5]

tf_vec = CountVectorizer(stop_words = stopWords, ngram_range = (1,1), max_features = 5000)
tf_vec.fit(train_all_articles_list + train_headlines_list)
#tf_vec.transform(train_articles_dict.values()).shape

tfidf_vec = TfidfVectorizer(stop_words = stopWords, ngram_range = (1,1), max_features = 5000, norm = 'l2')
tfidf_vec.fit(train_all_articles_list + train_headlines_list + test_all_articles_list + test_headlines_list)
#tfidf_vec.transform(train_articles_dict.values()).shape


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=5000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words=['a', 'about', 'above', 'across', 'after',
                            'afterwards', 'again', 'against', 'all', 'almost',
                            'alone', 'along', 'already', 'also', 'although',
                            'always', 'am', 'among', 'amongst', 'amoungst',
                            'amount', 'an', 'and', 'another', 'any', 'anyhow',
                            'anyone', 'anything', 'anyway', 'anywhere', ...],
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [11]:
train_hline_tf = tf_vec.transform(train_headlines_list)
test_hline_tf = tf_vec.transform(test_headlines_list)



train_hline_tfidf = tfidf_vec.transform(train_headlines_list)
test_hline_tfidf = tfidf_vec.transform(test_headlines_list)



print(train_hline_tf.shape)
print(test_hline_tf.shape)
print(train_hline_tfidf.shape)
print(test_hline_tfidf.shape)




train_article_list = []
test_article_list = []

for index, row in train_stances_df.iterrows():
    train_article_list.append(train_articles_df.iloc[train_dic_articleId_index[int(row['Body ID'])]]['articleBody'])
    #print(row['Body ID'])
    #print(train_articles_df.iloc[train_dic_articleId_index[int(row['Body ID'])]]['articleBody'])

for index, row in test_stances_df.iterrows():
    test_article_list.append(test_articles_df.iloc[test_dic_articleId_index[int(row['Body ID'])]]['articleBody'])

print(len(train_article_list))
print(len(test_article_list))



train_article_list_tf = tf_vec.transform(train_article_list)
test_article_list_tf = tf_vec.transform(test_article_list)

train_article_list_tfidf = tfidf_vec.transform(train_article_list)
test_article_list_tfidf = tfidf_vec.transform(test_article_list)
print(train_article_list_tf.shape)
print(test_article_list_tf.shape)
print(train_article_list_tfidf.shape)
print(test_article_list_tfidf.shape)

(49972, 5000)
(25413, 5000)
(49972, 5000)
(25413, 5000)
49972
25413
(49972, 5000)
(25413, 5000)
(49972, 5000)
(25413, 5000)


In [12]:
import numpy as np
from scipy.sparse import csr_matrix

cos_similarity_train = np.zeros(shape=(train_hline_tfidf.shape[0],1))
for x in range(train_hline_tfidf.shape[0]):
    A = csr_matrix.todense(train_hline_tfidf[x])
    B = csr_matrix.todense(train_article_list_tfidf[x])
    #print(train_hline_tf[x])
    cos_similarity_train[x] = np.dot(A,B.T)

print(cos_similarity_train[:5])

cos_similarity_test = np.zeros(shape=(test_hline_tfidf.shape[0],1))
for x in range(test_hline_tfidf.shape[0]):
    A = csr_matrix.todense(test_hline_tfidf[x])
    B = csr_matrix.todense(test_article_list_tfidf[x])
    #print(train_hline_tf[x])
    cos_similarity_test[x] = np.dot(A,B.T)

print(cos_similarity_test[:5])

[[0.        ]
 [0.40455841]
 [0.        ]
 [0.        ]
 [0.14184027]]
[[0.        ]
 [0.        ]
 [0.01324968]
 [0.        ]
 [0.        ]]


In [13]:
final_train_dense = np.zeros(shape=(train_hline_tfidf.shape[0],10001))
final_test_dense = np.zeros(shape=(test_hline_tfidf.shape[0],10001))

for x in range(train_hline_tfidf.shape[0]):
    A = csr_matrix.todense(train_hline_tfidf[x])
    B = csr_matrix.todense(train_article_list_tfidf[x])
    C = cos_similarity_train[x]
    #print(A.shape)
    #print(B.shape)
    #print(C.shape)
    row = np.squeeze(np.c_[A, B, C])
    final_train_dense[x] = row
    #print(row.shape)
    
print(final_train_dense.shape)


for x in range(test_hline_tfidf.shape[0]):
    A = csr_matrix.todense(test_hline_tfidf[x])
    B = csr_matrix.todense(test_article_list_tfidf[x])
    C = cos_similarity_test[x]
    #print(A.shape)
    #print(B.shape)
    #print(C.shape)
    row = np.squeeze(np.c_[A, B, C])
    final_test_dense[x] = row
    #print(row.shape)
    
print(final_test_dense.shape)

# t = hstack((hstack((train_hline_tf, cos_similarity_train)),train_article_list_tf))
# final_test = hstack((hstack((test_hline_tf, cos_similarity_test)),test_article_list_tf))
# print(final_train.shape)
# print(train_labels.shape)
# print(final_test.shape)
# print(test_labels.shape)

(49972, 10001)
(25413, 10001)


In [14]:
from sklearn.preprocessing import OneHotEncoder
print(train_labels.shape)
print(test_labels.shape)

print(train_labels)
train_encoder = OneHotEncoder(sparse=False)
train_labels = train_encoder.fit_transform(train_labels)
print(train_labels)


print(test_labels)
test_encoder = OneHotEncoder(sparse=False)
test_labels = test_encoder.fit_transform(test_labels)
print(test_labels)

(49972, 1)
(25413, 1)
[[3]
 [0]
 [3]
 ...
 [0]
 [2]
 [3]]
[[0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 ...
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[3]
 [3]
 [3]
 ...
 [1]
 [1]
 [0]]
[[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 ...
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]


/Users/rajbir/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/rajbir/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categori

In [56]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
model = keras.Sequential()
model.add(layers.Dense(100,input_dim=final_train_dense.shape[1], activation="relu"))
model.add(layers.Dropout(0.6))
model.add(layers.BatchNormalization())
model.add(layers.Dense(4, activation="softmax"))

opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_33 (Dense)             (None, 100)               1000200   
_________________________________________________________________
dropout_22 (Dropout)         (None, 100)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_34 (Dense)             (None, 4)                 404       
Total params: 1,001,004
Trainable params: 1,000,804
Non-trainable params: 200
_________________________________________________________________


In [57]:
model.fit(final_train_dense, train_labels, epochs=90, batch_size=500, verbose=0)
#evaluate the model
loss, accuracy = model.evaluate(final_test_dense, test_labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 86.872858


In [58]:
from score import report_score
ynew = model.predict_classes(final_test_dense)
print(ynew[:10])
predicted = [Labels[int(a)] for a in ynew]
print(predicted[:10])
y = test_stances_df['Stance'].tolist()
actual = [Labels[int(a)] for a in y]
print(actual[:10])
report_score(actual,predicted)

[3 3 3 3 3 3 3 3 3 3]
['unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated']
['unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated']
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    978    |     1     |    787    |    137    |
-------------------------------------------------------------
| disagree  |    300    |     5     |    281    |    111    |
-------------------------------------------------------------
|  discuss  |    758    |     5     |   3377    |    324    |
-------------------------------------------------------------
| unrelated |    184    |     4     |    444    |   17717   |
-------------------------------------------------------------
Score: 9322.25 out of 11651.25	(80

80.01072846261131